In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
subdirectory = Path('house-price')
df_train = pd.read_csv(subdirectory / 'dataset.csv')
df_test = pd.read_csv(subdirectory / 'test.csv')
print("No of training records: ", len(df_train))
print("No of features: ", len(df_train.columns))
#df_train.head()

In [ ]:
from house_price_features import feature_engineering, feature_clean_up
from house_price_optuna import run_winkler_predictor_optimization1

# Feature engineering

In [ ]:
df_train_fe=feature_engineering(df_train)
df_test_fe=feature_engineering(df_test)

print("Column dtypes:\n",df_train_fe.dtypes.value_counts())
#df_train_fe.columns

# Grid search for best Winkler score

In [ ]:
best_model, clusters, _, _, _ = run_winkler_predictor_optimization1(df_train_fe, n_trials=2, alpha=0.1)

In [ ]:
best_model.feature_importances().index.to_list()[:30]

# Submit results

In [ ]:

X_test = clusters.transform(df_test_fe)
X_test = feature_clean_up(X_test)

lower_bounds, upper_bounds = best_model.predict_interval(X_test)

result_df = pd.DataFrame({
        'id': df_test_fe['id'],
        'pi_lower': lower_bounds,
        'pi_upper': upper_bounds
    })

result_df.to_csv(subdirectory / 'submission.csv', index=False)